In [1]:
import xarray as xr
from datetime import datetime

import torch

from aurora import AuroraSmall, Batch, Metadata, rollout
import matplotlib.pyplot as plt

from pathlib import Path

import cdsapi
import numpy as np
from sklearn.metrics import root_mean_squared_error
import gcsfs

from torch.utils.data import Dataset
from aurora import Batch, Metadata
import os

/home/koomi_aims_ac_za/koomi/projects/AIMS-PROJECT/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import sys
sys.path.append(os.path.abspath("../src"))


In [6]:
def print_trainable_parameters(model):
    parameters, trainable = 0, 0
    
    for _, p in model.named_parameters():
        parameters += p.numel()
        trainable += p.numel() if p.requires_grad else 0
    print(trainable)
    print(f"trainable parameters: {trainable:,}/{parameters:,} ({100 * trainable / parameters:.2f}%)")

In [7]:
model = AuroraSmall(
    use_lora=False,  # Model was not fine-tuned.
    autocast=True,  # Use AMP.
)
model.load_state_dict(torch.load('../model/aurora-pretrained.pth'))

<All keys matched successfully>

In [8]:
print_trainable_parameters(model)

112797584
trainable parameters: 112,797,584/112,797,584 (100.00%)


In [9]:
for param in model.parameters():
    param.requires_grad = False

In [10]:
print_trainable_parameters(model)

0
trainable parameters: 0/112,797,584 (0.00%)


# Lora

In [11]:
from functools import partial
from lora import LinearWithLoRA
# default hyperparameter choices
lora_r = 8
lora_alpha = 16
# lora_dropout = 0.05
# lora_query = True
# lora_key = False
# lora_value = True
# lora_projection = False
# lora_mlp = False
# lora_head = False

# layers = []

assign_lora = partial(LinearWithLoRA, rank=lora_r, alpha=lora_alpha)

# Add lora to some parts

# Backbone

##  MLP

In [12]:
model.backbone.time_mlp[0] = assign_lora(model.backbone.time_mlp[0])
model.backbone.time_mlp[2] = assign_lora(model.backbone.time_mlp[2])

## Encoder

In [13]:
for block in model.backbone.encoder_layers:
    for layer in block.blocks:
        layer.norm1.ln_modulation[1] = assign_lora(layer.norm1.ln_modulation[1])
        layer.attn.qkv = assign_lora(layer.attn.qkv)
        layer.attn.proj = assign_lora(layer.attn.proj)
        layer.norm2.ln_modulation[1] =  assign_lora(layer.norm2.ln_modulation[1])
        layer.mlp.fc1 = assign_lora(layer.mlp.fc1)
        layer.mlp.fc2 = assign_lora(layer.mlp.fc2)
    if  block.downsample:
        block.downsample.reduction = assign_lora(block.downsample.reduction)

    


## Decoder

In [14]:
for block in model.backbone.decoder_layers:
    for layer in block.blocks:
        layer.norm1.ln_modulation[1] = assign_lora(layer.norm1.ln_modulation[1])
        layer.attn.qkv = assign_lora(layer.attn.qkv)
        layer.attn.proj = assign_lora(layer.attn.proj)
        layer.norm2.ln_modulation[1] =  assign_lora(layer.norm2.ln_modulation[1])
        layer.mlp.fc1 = assign_lora(layer.mlp.fc1)
        layer.mlp.fc2 = assign_lora(layer.mlp.fc2)
    if  block.upsample:
        block.upsample.lin1 = assign_lora(block.upsample.lin1)
        block.upsample.lin2 = assign_lora(block.upsample.lin2)

    


# Load training weights

In [22]:
checkpoint = torch.load("../model/checkpoints/checkpoint_epoch_20.pth")

In [ ]:
# model.load_state_dict(checkpoint)

In [23]:
model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


<All keys matched successfully>

In [25]:
torch.save(model.state_dict(), "../model/initial_model1.pth")

In [12]:
print_trainable_parameters(model)

1978368
trainable parameters: 1,978,368/114,775,952 (1.72%)


# Get south africa Data

In [13]:
fs = gcsfs.GCSFileSystem(token="anon")

store = fs.get_mapper('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
full_era5 = xr.open_zarr(store=store, consolidated=True, chunks=None)

In [14]:
start_time, end_time = '2023-01-10', '2023-01-31'


In [15]:

lat_max = -22.00 
lat_min = -37.75  

lon_min = 15.25   
lon_max = 35.00   
sliced_era5_SA = (
    full_era5
    .sel(
        time=slice(start_time, end_time),
        latitude=slice(lat_max, lat_min),
        longitude=slice(lon_min, lon_max)  
    )
)



# Trainig

In [17]:
from train import training
import torch.optim as optim

In [18]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [19]:
from loss import AuroraLoss

In [20]:
criterion = AuroraLoss()

In [21]:
model,  rmses = training(model=model, criterion=criterion,
             num_epochs=20, optimizer=optimizer,
             dataset= sliced_era5_SA,
             dataset_name="ERA5", 
             accumulation_steps=8,
             checkpoint_dir = '../model/checkpoints')

Epoch [1/20], Loss: 173.8280
Epoch [2/20], Loss: 155.3041
Epoch [3/20], Loss: 114.1772
Epoch [4/20], Loss: 84.3582
Epoch [5/20], Loss: 61.8708
Epoch [6/20], Loss: 48.3436
Epoch [7/20], Loss: 50.3122
Epoch [8/20], Loss: 49.6999
Epoch [9/20], Loss: 44.8104
Checkpoint saved: ../model/checkpoints/checkpoint_epoch_10.pth
Epoch [10/20], Loss: 41.3047
Epoch [11/20], Loss: 39.8103
Epoch [12/20], Loss: 38.3962
Epoch [13/20], Loss: 36.8970
Epoch [14/20], Loss: 35.9715
Epoch [15/20], Loss: 35.5212
Epoch [16/20], Loss: 34.8858
Epoch [17/20], Loss: 34.0254
Epoch [18/20], Loss: 32.9698
Epoch [19/20], Loss: 31.9503
Checkpoint saved: ../model/checkpoints/checkpoint_epoch_20.pth
Epoch [20/20], Loss: 31.2207


In [24]:
torch.save(model.state_dict(), "../model/best_models/best_model.pth")

In [ ]:
save_path  = "../report/training"
_, ax = plt.subplots(figsize=(8, 6), dpi=300)

ax.plot(np.arange(1,len(rmses)+1), rmses)
ax.set_ylabel("Mean Absolute Error (MAP)")
ax.set_xlabel("Epoch")
plt.savefig(f"{save_path}/map-learning-curve.pdf", bbox_inches="tight")
plt.savefig(f"{save_path}/map-learning-curve.png", bbox_inches="tight")
plt.savefig(f"{save_path}/map-learning-curve.svg", bbox_inches="tight")